In [1]:
import json
import numpy as np
import random
import pickle
import nltk
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD
from nltk.stem import WordNetLemmatizer


# Lemmatizer untuk memproses teks
lemmatizer = WordNetLemmatizer()

data_file = "data.json"
# Buka file dengan encoding 'utf-8'
with open(data_file, encoding="utf-8") as file:
    intents = json.load(file)

print("File berhasil dimuat!")

File berhasil dimuat!


In [2]:
# List kata unik, kelas, dan data pelatihan
words = []
classes = []
documents = []
ignore_words = ["?", "!", ".", ","]

# Tokenisasi dan lemmatization
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        # Tokenisasi
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent["tag"]))
        # Menambahkan kelas (tags)
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

# Lemmatization dan penghapusan duplikasi
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(set(words))
classes = sorted(set(classes))

# Simpan kata dan kelas menggunakan pickle
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))


In [3]:
# Membuat data pelatihan
training = []
output_empty = [0] * len(classes)

for doc in documents:
    # Bag of Words untuk pola
    bag = []
    word_patterns = [lemmatizer.lemmatize(w.lower()) for w in doc[0]]
    for word in words:
        bag.append(1 if word in word_patterns else 0)

    # Output (kelas target)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# Shuffle dan convert ke numpy array
random.shuffle(training)
training = np.array(training, dtype=object)

# Split fitur dan label
train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))


In [4]:
# Membangun model Sequential
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))

# Optimizer dan compile model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

# Train model
history = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
model.save("chatbot_model.h5")
print("Model telah dilatih dan disimpan!")


Epoch 1/200


C:\Users\fatur\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.0071 - loss: 2.4396    
Epoch 2/200
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1157 - loss: 2.3792  4
Epoch 3/200
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1976 - loss: 2.3360
Epoch 4/200
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2119 - loss: 2.2839
Epoch 5/200
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3309 - loss: 2.2346
Epoch 6/200
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2766 - loss: 2.1515
Epoch 7/200
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3582 - loss: 2.1077
Epoch 8/200
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3562 - loss: 2.0471
Epoch 9/200
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3616 - loss: 1.8963   
Epoch 10/200
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3841 - loss: 1.8652
Epoch 11/200
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3874 - loss: 1.8541
Epoch 12/200
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0

Model telah dilatih dan disimpan!
